马尔可夫过程
1. 状态空间
2. 无记忆性
3. 转移矩阵

Thought - Action - Observation的过程如何拆分并实现
1. 理解question要求，根据question决定使用哪个工具，如果都不满足则输出我不会（thought1）
2. 调用工具，将输入给到工具（action）
3. 工具返回结果（observation）
4. 判断结果是否完成question要求（thought2）
5. 如果完成则输出答案，如果未完成继续决定使用哪个工具（thought1）

In [36]:
import numpy as np
from datetime import date
import random
import re

In [4]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("C:\\Users\\By Yu\\chatglm-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("C:\\Users\\By Yu\\chatglm-6b", trust_remote_code=True).quantize(4).half().cuda()
model = model.eval()

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [126]:
# framework of Agent without method implementations
class Agent:
    def __init__(self, tool_list): # tool_list is a list of tuples with tool method and description
        # build tool dictionaries for functions and descriptions
        self.tool_method_dict = {}
        self.tool_description_dict = {}
        self.thought = [] # lists of dictionaries, each contains tool name, tool input, and reason to take the tool
        for tool in tool_list:
            name = tool[0].__name__ # get method name
            self.tool_method_dict[name] = tool[0]
            self.tool_description_dict[name] = tool[1]
            
    def __str__(self):
        return f"{self.tool_method_dict}\n{self.tool_description_dict}"
    
    def add_tools(self, tool_list):
        # add more tools to the agent after initialization
        for tool in tool_list:
            name = tool[0].__name__ # get method name
            if name not in self.tool_method_dict.keys(): # if method exist, don't replace
                self.tool_method_dict[name] = tool[0]
                self.tool_description_dict[name] = tool[1]
    
    def replace_tools(self, tool_list):
        # replace methods already exist, and add those not
        for tool in tool_list:
            name = tool[0].__name__ # get method name
            self.tool_method_dict[name] = tool[0]
            self.tool_description_dict[name] = tool[1]
    
    def call_tool(self, name, query):
        # find and call the specified tool
        if name not in self.tool_method_dict.keys():
            print("Tool not exist. Calling failed. Exit with -1.")
            return -1
        return self.tool_method_dict[name](query) # return a string, the output from the tool
    
    def thought1_decorator(self, func): # is there a matched tool, which tool to use
        def inner1(*args, **kwargs):
            return_value = func(*args, self.tool_description_dict, **kwargs)
            # return_value should be a list
            # [0, thought/inference, tool name, tool input query] or
            # [-1, thought/inference]
            if return_value[0] == -1:
                return_value[1] += "\nI don't know."
            return return_value
        return inner1
    
    def thought1_default(self, query=""):
        # default thought1 method
        prompt = f"""You are now a tool selector. \
Your job is to select one tool listed below that best performs the task described in the query, in addition to the tools you previously selected.
You should not answer the query by yourself.
Below is the query, the descriptions of all tools, the tools you've previously selected and their corresponding observations.

Query: {query}

Tool Descriptions:
"""
        tool_list = self.tool_description_dict.items()
        for description, i in zip(tool_list, np.arange(1, len(tool_list)+1)):
            prompt += "Tool " + str(i) + ":\n"
            prompt += "Name: " + description[0] + '\n' + "Description: " + description[1] + '\n\n'
        
        prompt += "Previous Observations:\n"
        if len(self.thought) == 0:
            prompt += "None.\n"
        for d, i in zip(self.thought, np.arange(1,len(self.thought)+1)):
            prompt += "Tool " + str(i) + ": " + d.get("Name") + '\n'
            prompt += "Observation " + str(i) + ": " + str(d.get("Observation")) + '\n'
        
        prompt += """\nYour response has only three lines: the tool name, the input that tool takes, and the reason why you choose this tool. \
The input should have the format defined by the tool description.
Please respond in the format:
\"Name:
Input:
Reason: \""""
        # If none of the tools match the query, you should respond \"I don't know.\"
        print(prompt)
        response, history = model.chat(tokenizer, prompt, history=[])
        return response
        
        """
        for description in self.tool_description_dict.items():
            if description[1] in query:
                name = self.tool_method_dict.get(description[0]).__name__
                return [0, "yes! " + name, name, query]
        return [-1, "No matches for this function, bye!"]
        """
    
    def thought2_decorator(self, func):
        def inner2(*args, **kwargs):
            return_value = func(*args, **kwargs)
            return return_value
            # return_value should be a list
            # [0, thought/inference, answer] or
            # [1, thought/inference, final answer]
        return inner2
    
    def thought2_default(self, query):
        # default thought2 method
        prompt = f"""You are now a judge of several observations. \
Your job is to decide whether the observations listed below are sufficient to answer the question described in the query.
Below is the query, the tools you used, and the corresponding obervations.

Query: {query}

Tools and Observations:
"""

        for d, i in zip(self.thought, np.arange(1,len(self.thought)+1)):
            prompt += "Tool " + str(i) + ": " + d.get("Name") + '\n'
            prompt += "Observation " + str(i) + ": " + str(d.get("Observation")) + '\n'

        prompt += """\nYour response has only one line: an answer.\
If the observations are good enough to meet the query's need, \
your response should be \"good\"; if the observations are not good enough, your response should be \"continue\".
Please respond in the format:
\"Answer: \""""
        
        print(prompt)
        response, history = model.chat(tokenizer, prompt, history=[])
        return response
    
    def output(self):
        final_output = ""
        for d in self.thought:
            final_output += str(d.get("Observation")) + '\n'
        return final_output
    
    """
    def concatenate(self, query, inference, observation):
        # combine the query, inferences and observations into one new prompt
        prompt = query + '\n\n'
        for i in range(0, len(inference)-1, 2):
            prompt += inference[i] + '\n' + str(observation[i//2]) + '\n' + inference[i+1] + '\n\n'
        return prompt
    """
    
    def run(self, query, thought1 = thought1_default, thought2 = thought2_default):
        self.thought = []
        i = 1
        while True:
            # thought 1
            if thought1.__name__ == self.thought1_default.__name__:
                self.thought.append(str_to_dict(thought1(self, query=query))) 
            else:
                self.thought.append(str_to_dict(thought1(query=prompt)))
            print("\nthought {}\n{}".format(i, self.thought[-1]), end='\n\n')
            
            # action
            """
            if self.thought[i][0] == -1:
                return self.output(final_output=self.thought[i][2]) # I don't know
            
            """
            observation = self.call_tool(name=self.thought[-1].get("Name"), query=self.thought[-1].get("Input"))
            self.thought[-1]["Observation"] = observation
            # print("observation", i//2, self.observation[-1])
            
            # thought 2
            self.thought[-1]["Answer"] = ''
            while self.thought[-1].get("Answer") == '': # continues to judge if no output answer
                if thought2.__name__ == self.thought2_default.__name__:
                    answer = thought2(self, query=query)
                    print(answer)
                    answer = str_to_dict(answer)
                    self.thought[-1]["Answer"] = answer.get("Answer")
                    print("\nthought {}\n{}".format(i, self.thought[-1]), end='\n\n')
                else:
                    answer = str_to_dict(thought2(query=query))
                    self.thought[-1]["Answer"] = answer.get("Answer")
                    print("\nthought {}\n{}".format(i, self.thought[-1]), end='\n\n')
                
            if "continue" not in self.thought[-1].get("Answer").lower(): # exit if final answer is given
                break
            i += 1
        
        return self.output()
        # output(self.thought[-1][-1]) # output final answer
        
        
        

In [121]:
def find_date(query=""):
    return str(date.today())

find_date_description = """Returns todays date, use this for any \
questions related to knowing todays date. \
The input should always be an empty string, \
and this function will always return todays \
data - any date mathmatics should occur \
outside this function."""
print(find_date_description)

Returns todays date, use this for any questions related to knowing todays date. The input should always be an empty string, and this function will always return todays data - any date mathmatics should occur outside this function.


In [129]:
def gen_rand(query = "01"):
    num_list = re.findall('\d+', query)
    if len(num_list) < 2:
        return float("-inf")
    a = int(min(num_list[0], num_list[1]))
    b = int(max(num_list[0], num_list[1]))
    return random.randint(a, b)

gen_rand_description = """Return a random integer, use this for any \
query for a random value within a given range.\
The input is a pair of two integers, representing \
the random bound, and the two integers are given by the query. \
This function will always return a randomly generated integer.
"""

In [123]:
print(find_date())
print(gen_rand("Give me a random number between 1 and 10."))

2023-07-28
4


In [130]:
agent = Agent(tool_list=[(find_date, find_date_description)])
agent.add_tools(tool_list=[(gen_rand, gen_rand_description)])

In [133]:
answer = agent.run(query="Give me a random integer between (3, 9).")
print(f"\nAnswer\n{answer}")

You are now a tool selector. Your job is to select one tool listed below that best performs the task described in the query, in addition to the tools you previously selected.
You should not answer the query by yourself.
Below is the query, the descriptions of all tools, the tools you've previously selected and their corresponding observations.

Query: Give me a random integer between (3, 9).

Tool Descriptions:
Tool 1:
Name: find_date
Description: Returns todays date, use this for any questions related to knowing todays date. The input should always be an empty string, and this function will always return todays data - any date mathmatics should occur outside this function.

Tool 2:
Name: gen_rand
Description: Return a random integer, use this for any query for a random value within a given range.The input is a pair of two integers, representing the random bound, and the two integers are given by the query. This function will always return a randomly generated integer.


Previous Obser

In [56]:
response = agent.thought1_default(query="I want to know what date is it today.")
print(f"\nResponse\n{response}")

You are now a tool selector. Your job is to select one tool listed below that best performs the task described in the query, in addition to the tools you previously selected.
You should not answer the query by yourself.
Below is the query, the descriptions of all tools, the tools you've previously selected and their corresponding observations.

Query: I want to know what date is it today.

Tool Descriptions:
Tool 1:
Name: find_date
Description: Returns todays date, use this for any questions related to knowing todays date. The input should always be an empty string, and this function will always return todays data - any date mathmatics should occur outside this function.

Tool 2:
Name: gen_rand
Description: Return a random integer, use this for any query for a random value within a given range.The input should always be a string with two integers representing the bound required in the query, and the two integers are separated with a comma, and this function will always return a randoml

In [95]:
def str_to_dict(string):
    items = string.split('\n')
    dictionary = {}
    for i in items:
        key_value = i.split(':')
        # print("key_value:", key_value)
        if key_value[1] == '':
            dictionary[key_value[0]] = ''
        elif key_value[1][0] != " " or len(key_value[1]) == 1:
            dictionary[key_value[0]] = key_value[1]
        else:
            dictionary[key_value[0]] = key_value[1][1:]
    return dictionary

In [84]:
response = """Name: find_date
Input:
Reason: This tool will always return today's date."""
d = str_to_dict(response)
print(d.items())

dict_items([('Name', 'find_date'), ('Input', ''), ('Reason', "This tool will always return today's date.")])


In [41]:
@ agent.thought2_decorator
def random_exit(query=""):
    query_list = query.split('\n')
    if random.random() > 0.5:
        return [0, f"Please continue\n{query_list[0]}", query_list[2]]
    return [1, "Finished!", query_list[2]]

In [166]:
result = agent.run("Give me a random number between 1 and 10", thought2=random_exit)
print(result)

thought 0 [0, 'yes! gen_rand', 'gen_rand', 'Give me a random number between 1 and 10']
inference 0 yes! gen_rand
observation 0 2
thought 1 [0, 'Please continue\nGive me a random number between 1 and 10', '']
inference 1 Please continue
Give me a random number between 1 and 10
thought 2 [0, 'yes! gen_rand', 'gen_rand', 'Give me a random number between 1 and 10\n\nyes! gen_rand\n2\nPlease continue\nGive me a random number between 1 and 10\n\n']
inference 2 yes! gen_rand
observation 1 6
thought 3 [1, 'Finished!', 'yes! gen_rand']
inference 3 Finished!
2


In [152]:
print(result)

4


In [33]:
if "random" in "aijsdifj randomnesss":
    print("ah")

ah


Test class initialization

In [38]:
def abc():
    print("hello")
dic = {"a": abc}
dic.get("a")()

hello


Test class \_\_str\_\_()

In [75]:
a = Agent([(abc, "asuhuhdd")])

In [76]:
print(a)

{'abc': <function abc at 0x000002A724CBD9D0>}
{'abc': 'asuhuhdd'}


Test in-class decorator function calling class variables

In [77]:
@ a.thought1
def print_lahoho(a, b, c):
    print("lahoho")
    print("a: ", a)
    print("b: ", b)
    print("c: ", c)
    return 0

In [80]:
print_lahoho(3, c=1)

{'abc': <function abc at 0x000002A724CBD9D0>}
lahoho
a:  3
b:  {'abc': 'asuhuhdd'}
c:  1


0

Testing decorator

In [168]:
class Person:
    def __init__(self):
        self.name = "Michael"
    
    def dec(func):
        def inner(*args, **kwargs):
            return_value = func(*args, **kwargs)
            return return_value
        return inner
    
    @ dec
    def lulula(self, name):
        print(self.name)
        print(name)
        return 0
    
    def call_name(self, name, func=lulula):
        if func.__name__ == self.lulula.__name__:
            print("hello lulula")
            func(self, name)
        else:
            func(name)

In [169]:
def polite(name="Peter"):
    print(f"Hi, my name is {name}.")

In [170]:
michael = Person()
michael.lulula("Gloria")
michael.call_name(name="Peter", func=polite)
michael.call_name(name="Jack")

Michael
Gloria
Hi, my name is Peter.
hello lulula
Michael
Jack


In [37]:
def dectest(func):
    def inner(*args, **kwargs):
        return_value = func(*args, **kwargs)
        return return_value
    return inner

In [81]:
def print_holala():
    print("holala")
    return 0

In [83]:
@ dectest
print_holala

SyntaxError: invalid syntax (266413596.py, line 2)

In [3]:
test = """You are now asked to perform one or many tasks with the tools you have. 
Below is the query and the tool description"""
print(test)

You are now asked to perform one or many tasks with the tools you have. 
Below is the query and the tool description


In [104]:
print(float("-inf")+2)

-inf
